# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="./Tennis_Linux/Tennis.x86_64")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [5]:
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

Score (max over agents) from episode 1: 0.09000000171363354
Score (max over agents) from episode 2: 0.0
Score (max over agents) from episode 3: 0.10000000149011612
Score (max over agents) from episode 4: 0.0
Score (max over agents) from episode 5: 0.09000000171363354


When finished, you can close the environment.

In [7]:
# env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [14]:
import random
from collections import deque, namedtuple

class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed, device):
        """Initialize a ReplayBuffer object.
        Params
        ======
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  # internal memory (deque)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
        self.device = device
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(self.device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float().to(self.device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(self.device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(self.device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)

        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

In [8]:
class OUNoise:
    """Ornstein-Uhlenbeck process."""

    def __init__(self, size, seed, mu=0., theta=0.15, sigma=0.1):
        """Initialize parameters and noise process."""
        self.mu = mu * np.ones(size)
        self.theta = theta
        self.sigma = sigma
        self.seed = random.seed(seed)
        self.reset()

    def reset(self):
        """Reset the internal state (= noise) to mean (mu)."""
        self.state = copy.copy(self.mu)

    def sample(self):
        """Update internal state and return it as a noise sample."""
        x = self.state
        # dx = self.theta * (self.mu - x) + self.sigma * np.array([random.random() for i in range(len(x))])
        dx = self.theta * (self.mu - x) + self.sigma * np.random.randn(len(x))
        self.state = x + dx
        return self.state

In [9]:
# https://github.com/ikostrikov/pytorch-ddpg-naf/blob/master/ddpg.py#L11
def soft_update(target, source, tau):
    """
    Perform DDPG soft update (move target params toward source based on weight
    factor tau)
    Inputs:
        target (torch.nn.Module): Net to copy parameters to
        source (torch.nn.Module): Net whose parameters to copy
        tau (float, 0 < x < 1): Weight factor for update
    """
    for target_param, param in zip(target.parameters(), source.parameters()):
        target_param.data.copy_(target_param.data * (1.0 - tau) + param.data * tau)

# https://github.com/ikostrikov/pytorch-ddpg-naf/blob/master/ddpg.py#L15
def hard_update(target, source):
    """
    Copy network parameters from source to target
    Inputs:
        target (torch.nn.Module): Net to copy parameters to
        source (torch.nn.Module): Net whose parameters to copy
    """
    for target_param, param in zip(target.parameters(), source.parameters()):
        target_param.data.copy_(param.data)

def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1. / np.sqrt(fan_in)
    return (-lim, lim)

In [6]:
import torch
import torch.nn as nn
import numpy as np


class Network(nn.Module):
    def __init__(
        self, 
        in_size: int,
        out_size: int,
        fc1: int=256,
        fc2: int=128,
        is_actor: bool=True) -> None:

        super().__init__()
        self.fc1 = nn.Linear(in_size, fc1)
        self.fc2 = nn.Linear(fc1, fc2)
        self.fc3 = nn.Linear(fc2, out_size)
        self.is_actor = is_actor
    
    def reset_parameters(self):
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-1e-3, 1e-3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        if self.is_actor:
            return torch.tanh(self.fc3(x))
        else:
            return self.fc3(x)

In [11]:
import torch.optim as optim

class DDPGAgent:
    def __init__(
        self,
        in_actor: int,
        out_actor: int,
        in_critic: int,
        random_seed: int,
        lr_actor: float = 5e-4,
        lr_critic: float = 5e-4,
        device: str = "cpu"
        ) -> None:
        
        self.actor = Network(in_actor, out_actor, is_actor=True).to(device)
        self.critic = Network(in_critic, 1, is_actor=False).to(device)
        self.actor_target = Network(in_actor, out_actor, is_actor=True).to(device)
        self.critic_target = Network(in_critic, 1, is_actor=False).to(device)

        self.noise = OUNoise(out_actor, random_seed)

        hard_update(self.actor_target, self.actor)
        hard_update(self.critic_target, self.critic)

        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=lr_actor)
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=lr_critic, weight_decay=1.e-5)

        self.device = device
        self.random_seed = random_seed

    def act(self, obs, noise=0.0):
        obs = obs.to(self.device)
        action = self.actor(obs) + noise * self.noise.sample()
        return np.clip(action, -1, 1)

    def target_act(self, obs, noise=0.0):
        obs = obs.to(self.device)
        action = self.actor_target(obs) + noise * self.noise.sample()
        return np.clip(action, -1, 1)


In [15]:
class MADDPG:
    def __init__(
        self, 
        in_actor: int,
        out_actor: int,
        in_critic: int,
        random_seed: int,
        buffer_size: int=int(1e6),
        batch_size: int=128,
        gamma :float=0.95, 
        tau :float=0.02,
        device: str = "cpu"
        ) -> None:

        # For each agent in the environment
        self.maddpg_agents = [
            DDPGAgent(in_actor, out_actor, in_critic, random_seed=random_seed, device=device),
            DDPGAgent(in_actor, out_actor, in_critic, random_seed=random_seed, device=device)
        ]

        self.memory = ReplayBuffer(out_actor, buffer_size, batch_size, random_seed, device)

        self.gamma = gamma
        self.tau = tau
        self.device = device

    def get_actors(self):
        return [agent.actor for agent in self.maddpg_agents]

    def get_target_actors(self):
        return [agent.target_act for agent in self.maddpg_agents]

    def act(self, obs_all_agents, noise=0.0):
        return [agent.act(obs, noise) for agent, obs in zip(self.maddpg_agents, obs_all_agents)]

    def target_act(self, obs_all_agents, noise=0.0):
        return [agent.target_act(obs, noise) for agent, obs in zip(self.maddpg_agents, obs_all_agents)]


    def step(self, state, action, reward, next_state, done):
        """Save experience in replay memory, and use random sample from buffer to learn."""
        # Save experience / reward
        self.memory.add(state, action, reward, next_state, done)

        # Learn, if enough samples are available in memory
        if len(self.memory) > self.batch_size:
            experiences = self.memory.sample()
            for i in range(len(self.maddpg_agents)):
                self.learn(experiences, i)
            self.update_targets()
            

    def learn(self, samples, agent_number):

        obs, obs_full, action, reward, next_obs, next_obs_full, dones = samples
        
        agent = self.maddpg_agents[agent_number]
        agent.critic_optimizer.zero_grad()
        target_actions = self.target_act(next_obs)
        target_actions = torch.cat(target_actions, dim=1)

        target_critic_input = torch.cat((next_obs_full, target_actions), dim=1).to(self.device)

        with torch.no_grad():
            q_next = agent.critic_target(target_critic_input)

        y = reward[agent_number].view(-1, 1) + self.gamma * q_next * (1 - dones[agent_number].view(-1, 1))
        critic_input = torch.cat((obs_full, action), dim=1).to(self.device)
        q = agent.critic(critic_input)

        huber_loss = torch.nn.SmoothL1Loss()
        critic_loss = huber_loss(q, y.detach())
        critic_loss.backward()
        agent.critic_optimizer.step()

        agent.actor_optimizer.zero_grad()
        q_input = []
        for i, ob in enumerate(obs):
            if i == agent_number:
                q_input.append(self.maddpg_agents[i].actor(ob))
            else:
                q_input.append(self.maddpg_agents[i].actor(ob).detach())

        q_input = torch.cat(q_input, dim=1)
        q_input = torch.cat(obs_full, q_input, dim=1)

        actor_loss = -agent.critic(q_input).mean()
        actor_loss.backward()
        agent.actor_optimizer.step()

    def update_targets(self):
        """soft update targets"""
        self.iter += 1
        for ddpg_agent in self.maddpg_agents:
            soft_update(ddpg_agent.target_actor, ddpg_agent.actor, self.tau)
            soft_update(ddpg_agent.target_critic, ddpg_agent.critic, self.tau)


        

In [13]:
from tqdm import tqdm

def maddpg(
    driver :MADDPG,
    env,
    n_episodes=1000, 
    max_t=300):
    scores_window = deque(maxlen=100)
    scores = []
    with tqdm(total=n_episodes) as progress:
        for i_episode in range(1, n_episodes+1):
            env_info = env.reset(train_mode=True)[brain_name]
            
            states = env_info.vector_observations
            num_agents = len(env_info.agents)
            score = np.zeros(num_agents)
            
            for t in range(max_t):
                actions = driver.act(states)
                env_info = env.step(actions)[brain_name]
                next_states = env_info.vector_observations
                rewards = env_info.rewards
                rewards = np.expand_dims(np.asanyarray(rewards), axis=1)
                dones = env_info.local_done
                dones = np.expand_dims(np.asanyarray(dones), axis=1)
    
                driver.step(states, actions, rewards, next_states, dones)
                states = next_states
                score += np.squeeze(rewards)
                if np.any(dones):
                    break
                
            score = np.max(score) 
            scores_window.append(score)
            scores.append(score)
            
            progress.set_postfix({"Avg. Score": f"{np.mean(scores_window):.2f}"})
            progress.update()

            if i_episode >=100 and np.mean(scores_window) >= 30.0:
                print(f"Environment solved at {i_episode} episodes with Avg. score: {np.mean(scores_window):.2f}")
                for i in range(num_agents):
                    torch.save(driver.maddpg_agents[i].actor.state_dict(), f'./cp/agent-{i}-checkpoint_actor.pth')
                    torch.save(driver.maddpg_agents[i].critic.state_dict(), f'./cp/agent-{i}-checkpoint_critic.pth')
                break
            
    return scores